In [23]:
%cd turnus

d:\Documents\experimenty\rl-optimization\turnus


In [78]:
import torch
import pandas as pd
import numpy as np

def load_node_csv(path: str, index_col: str, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)
        df = df[encoders.keys()].copy()

    return x, mapping, df

def build_edge_connections(time_matrix_path: str, finish_times: np.array, start_times: np.array, delimiter=';'):
    matrix = np.loadtxt(time_matrix_path, delimiter=delimiter)
    # Add the transport times (matrix) to the finishing time of every task
    transport_times = matrix + np.expand_dims(finish_times, axis=-1)

    possible_connection = transport_times <= start_times

    return np.where(possible_connection)


In [59]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [64]:
# Normalize and convert dataframe column (Series) into a tensor
class NumberNormEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(((df - df.mean()) / df.std()).values).view(-1, 1).to(self.dtype)

In [ ]:
from torch_geometric.data import Data

nodes, mapping, df = load_node_csv('data/1/tasks.csv', 'Index', 
                               {name: NumberNormEncoder() for name in ['ZastavkaStart', 'ZastavkaFinish', 'CasStart', 'CasFinish', 'Vzdialenost']}, 
                               sep=';')

edge_connections = build_edge_connections('data/1/Tij.csv', df)

data = Data(x=nodes)

df, nodes

In [139]:
matrix = np.loadtxt('data/1/Tij.csv', delimiter=';')

transport_times = matrix + np.expand_dims(df['CasFinish'].values, axis=-1)

possible_connection = transport_times <= df['CasStart'].values

possible_connection[-1], [possible_connection[-1]])

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True]),
 (array([0], dtype=int64), array([50], dtype=int64)))

In [7]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [16]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [19]:
data.num_edge_features

0

In [10]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8180
